#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "pbXLVO0CmR6d66hRTpb6Lt8H4q14f3RF"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [1]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
0. inputとoutputはビジネスに関連した内容を生成してください．
1. outputはビジネス現場で用いられるような適切な表現で生成してください．
2. inputとoutoputは一般的に知られている内容を生成してください．特定の企業の内容やプロジェクト名などを用いてはいけません．
3. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
4. 出力は指示に対する適切な応答であるべきです。出力はinputの回答となっていることを確認してください．
5. 生成するinputとoutputは全て必ず日本語で生成してください。
6. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "貴方はデータアーキテクトです．質問に対して解答してください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  },
  {
    "id": 2,
    "instruction": "あなたは先輩社員です．後輩社員からの質問に丁寧に回答してください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  },
  {
    "id": 3,
    "instruction": "貴方は企業で働くビジネスマンです．以下の質問に答えてください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  }
]
"""

In [2]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": "貴方はデータアーキテクトです．質問に対して解答してください．",
    "input": "データのサイロ化とは？",
    "output": "企業内でデジタルデータの保管場所が分散しデータの連携がされていない，もしくはできない状態のことを指します．"
  },
  {
    "id": 2,
    "instruction": "あなたは先輩社員です．後輩社員からの質問に丁寧に回答してください．",
    "input": "デジタイゼーションの具体的な方法は紙の文書をスキャンしOCR技術によってコンピュータに認識できるデータにすることです．それではデジタライゼーションの具体的な方法は何ですか？",
    "output": "デジタライゼーションの具体的な方法は，今まで人が行っている業務を自動化するためにシステムを開発することや，RPAによってロボット化することです．"
  },
  {
    "id": 3,
    "instruction": "貴方は企業で働くビジネスマンです．以下の質問に答えてください．",
    "input": "PDCAとは？",
    "output": "PDCAはPlan（計画），Do（実行），Check（測定・評価），Action（対策・改善）の頭文字をとったもので，これらのプロセスを回すことによって，マネジメントの品質を高めようという概念のことです．"
  },
  {
    "id": 4,
    "instruction": "貴方は企業で働くビジネスマンです．以下の質問に答えてください．",
    "input": "クラウドファンディングのメリットとデメリットを述べてください．",
    "output": """クラウドファンディングのメリットとして以下の点があげられます．
- 起案者の観点では資金調達の可能性が高まること
- 起案したアイデアが市場に出る前に，ユーザーの反応を知ることができること
- 支援者の観点では，一般のサービス取引と比較して作り手の顔が見えることによるコミュニケーションをとることができること

クラウドファンディングのデメリットとして以下の点があげられます．
- 起案者の観点では目標金額を調達できない可能性があること
- 支援者の観点では想定したリターンが得られないこと"""
  },
  {
    "id": 5,
    "instruction": "あなたはコンサルタント企業の社員です．質問に対して答えてください．",
    "input": "スタートアップ企業とユニコーン企業の定義と何が違うのか教えてほしいです",
    "output": "スタートアップ企業は「社会にイノベーションを起こし、新たな市場を創造、そして短期間に急激に成長する企業」のことで，ユニコーン企業は「評価額10億ドル以上・設立10年以内の未上場ベンチャー企業」のことです．それぞれの様々な観点で違いが存在し，ビジネスモデルとして革新的か否か，成長プランとして急激な成長と独占を目指すのか否かといった観点などで違いが存在します．"
  }
]

In [5]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)
print(formatted_prompt_template)


以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
0. inputとoutputはビジネスに関連した内容を生成してください．
1. outputはビジネス現場で用いられるような適切な表現で生成してください．
2. inputとoutoputは一般的に知られている内容を生成してください．特定の企業の内容やプロジェクト名などを用いてはいけません．
3. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
4. 出力は指示に対する適切な応答であるべきです。出力はinputの回答となっていることを確認してください．
5. 生成するinputとoutputは全て必ず日本語で生成してください。
6. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
[{'id': 1, 'instruction': '貴方はデータアーキテクトです．質問に対して解答してください．', 'input': 'データのサイロ化とは？', 'output': '企業内でデジタルデータの保管場所が分散しデータの連携がされていない，もしくはできない状態のことを指します．'}, {'id': 2, 'instruction': 'あなたは先輩社員です．後輩社員からの質問に丁寧に回答してください．', 'input': 'デジタイゼーションの具体的な方法は紙の文書をスキャンしOCR技術によってコンピュータに認識できるデータにすることです．それではデジタライゼーションの具体的な方法は何ですか？', 'output': 'デジタライゼーションの具体的な方法は，今まで人が行っている業務を自動化するためにシステムを開発することや，RPAによってロボット化することです．'}, {'id': 3, 'instruction': '貴方は企業で働くビジネスマンです．以下の質問に答えてください．', 'input': 'PDCAとは？', 'output': 'PDCAはPlan（計画），Do（実行），Check（測定・評価），Action（対策・改善）の頭文字をとったもので，これらのプロセスを回すことによって

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=10000
LOOP_NUM = 100 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

ストリーミング出力は最後の 5000 行に切り捨てられました。
  },
  {
    "id": 2,
    "instruction": "貴方は経営戦略担当です．質問に対して解答してください．",
    "input": "ブルーオーシャン戦略とは？",
    "output": "ブルーオーシャン戦略は、既存の市場で競合する代わりに新しい市場を創造し、新たな需要を作り出すことによって成長を目指す経営戦略のことです。"
  },
  {
    "id": 3,
    "instruction": "貴方は企業の開発担当者です．質問に対して解答してください．",
    "input": "アジャイル開発とは？",
    "output": "アジャイル開発とは、開発プロセス全体を小さなイテレーションに分割し、短期間ごとに品質を確認することで効率的かつ迅速にソフトウェア開発を行うための開発方法論のことです。"
  },
  {
    "id": 4,
    "instruction": "貴方はHR担当者です．質問に対して解答してください．",
    "input": "人材育成とは？",
    "output": "人材育成とは、社員や組織が今後必要とする能力を身に着けるために教育・研修や職場における経験を通じて学び、育成を行うことです。"
  },
  {
    "id": 5,
    "instruction": "貴方はIT業界の専門家です．質問に対して解答してください．",
    "input": "クラウドサービスとは？",
    "output": "クラウドサービスとは、インターネット上にあるサーバーを利用して、ユーザーが必要な場合に限りリソース（データやアプリケーションなど）を提供するようなサービスのことです。"
  },
  {
    "id": 6,
    "instruction": "貴方はビジネスモデルの専門家です．質問に対して解答してください．",
    "input": "レバレッジ効果とは？",
    "output": "レバレッジ効果とは、自社資本以外の資金を活用したときに得られるメリットのことで、例えば借金などの他社資本を活用して企業の収益性を向上させることがあります。"
  },
 

,id,instruction,input,output
0,1,貴方はエンジニアリングマネージャーです．スタートアップ企業の設立から8年が経過しており，開発...,スクラム法とは何ですか？それは何を図るものですか？,スクラム法とは，アジャイルソフトウェア開発の一つで，製品開発の効率化と品質向上を目的に，短期...
1,2,貴方はトレーナーです．社員教育プログラムにおいて，以下の質問に答えてください．,ある社員があなたと同じ部署から新しい部署へ移動しようとしています．その際，新たな部署での環境...,まずは新たな部署の目標やミッションを確認し，自分が監督されている業務の大きな課題を理解するこ...
2,3,貴方はビジネスマネジメントコンサルタントです．以下の質問に答えてください．,ITスタッフとして，顧客からの要望に対応する際に，要求が見積もりの範囲外であった場合はどのよ...,まずは顧客の要望について専門的に説明し，その要求が現在の仕様やスケジュールの範囲外であること...
3,4,貴方はデータサイエンティストです．以下の質問に答えてください．,機械学習モデルの開発にあたって，モデルの精度を上げるために重要なことは何ですか？,機械学習モデルの開発にあたっては，学習データやアルゴリズムの選択など，多岐にわたる要素が影響...
4,5,貴方は経営コンサルタントです．以下の質問に答えてください．,ビジネスモデルやビジネスプランを立案する際には，市場の要件や競合力を認識するためにどのような...,市場の要件や競合力を認識するためには，最初にターゲットマーケットを把握する必要があります．そ...


##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /generated_data_2024-05-18_10-58-31.xlsx
